# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [17]:
import re
from collections import defaultdict

def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

# Define the keyboard adjacency dictionary.
keyboard_adjacent = {
    'q': ['w', 'a', 's'],
    'w': ['q', 'e', 'a', 's', 'd'],
    'e': ['w', 'r', 's', 'd', 'f'],
    'r': ['e', 't', 'd', 'f', 'g'],
    't': ['r', 'y', 'f', 'g', 'h'],
    'y': ['t', 'u', 'g', 'h', 'j'],
    'u': ['y', 'i', 'h', 'j', 'k'],
    'i': ['u', 'o', 'j', 'k', 'l'],
    'o': ['i', 'p', 'k', 'l'],
    'p': ['o', 'l'],
    'a': ['q', 'w', 's', 'z', 'x'],
    's': ['a', 'd', 'q', 'w', 'e', 'z', 'x', 'c'],
    'd': ['s', 'f', 'w', 'e', 'r', 'x', 'c', 'v'],
    'f': ['d', 'g', 'e', 'r', 't', 'c', 'v', 'b'],
    'g': ['f', 'h', 'r', 't', 'y', 'v', 'b', 'n'],
    'h': ['g', 'j', 't', 'y', 'u', 'b', 'n', 'm'],
    'j': ['h', 'k', 'y', 'u', 'i', 'n', 'm'],
    'k': ['j', 'l', 'u', 'i', 'o', 'm'],
    'l': ['k', 'i', 'o', 'p'],
    'z': ['a', 's', 'x'],
    'x': ['z', 'c', 'a', 's', 'd'],
    'c': ['x', 'v', 's', 'd', 'f'],
    'v': ['c', 'b', 'd', 'f', 'g'],
    'b': ['v', 'n', 'f', 'g', 'h'],
    'n': ['b', 'm', 'g', 'h', 'j'],
    'm': ['n', 'h', 'j', 'k']
}

def keyboard_edits(word):
    """
    Generate candidate words by replacing each character with its adjacent keys.
    """
    candidates = set()
    for i, char in enumerate(word):
        if char in keyboard_adjacent:
            for adj in keyboard_adjacent[char]:
                candidate = word[:i] + adj + word[i+1:]
                candidates.add(candidate)
    return candidates

class NGramModel:
    def __init__(self):
        self.unigrams = defaultdict(int)
        self.bigrams = defaultdict(int)
        
    def load_data(self, bigram_path, fivegram_path, coca_path, big_txt_path):
        # Load bigrams from the bigram file (ASCII encoding)
        with open(bigram_path, 'r', encoding='ascii', errors='replace') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 3:
                    try:
                        count = int(parts[0])
                        w1 = parts[1].lower()
                        w2 = parts[2].lower()
                        self.unigrams[w1] += count
                        self.unigrams[w2] += count
                        self.bigrams[(w1, w2)] += count
                    except (ValueError, IndexError):
                        continue

        # Load fivegrams (extracting the first five words after the count)
        with open(fivegram_path, 'r', encoding='ascii', errors='replace') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 6:
                    try:
                        count = int(parts[0])
                        words = [w.lower() for w in parts[1:6]]
                        for word in words:
                            self.unigrams[word] += count
                        for i in range(len(words)-1):
                            self.bigrams[(words[i], words[i+1])] += count
                    except (ValueError, IndexError):
                        continue

        # Load coca_all_links data (ASCII encoding)
        with open(coca_path, 'r', encoding='ascii', errors='replace') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 3:
                    try:
                        count = int(parts[0])
                        w1 = parts[1].lower()
                        w2 = parts[2].lower()
                        self.unigrams[w1] += count
                        self.unigrams[w2] += count
                        self.bigrams[(w1, w2)] += count
                    except (ValueError, IndexError):
                        continue

        # Load big.txt and extract unigrams and bigrams from its normal text.
        with open(big_txt_path, 'r', encoding='utf-8') as f:
            for line in f:
                words_in_line = re.findall(r'\w+', line.lower())
                for word in words_in_line:
                    self.unigrams[word] += 1
                for i in range(len(words_in_line)-1):
                    self.bigrams[(words_in_line[i], words_in_line[i+1])] += 1

    def get_candidates(self, word):
        """
        Generate candidate corrections in a staged manner:
          1. If the word is known, return it.
          2. Else, try one-edit candidates (edits1 and keyboard_edits).
          3. If still no known candidate, try two-edits (edits2).
        """
        # If the original word is known, return it directly.
        if word in self.unigrams:
            return {word}
        
        # First try one-edit candidates and keyboard candidates.
        candidates = {w for w in edits1(word) if w in self.unigrams}
        candidates |= {w for w in keyboard_edits(word) if w in self.unigrams}
        
        # Only if no candidates found from one-edit, try two-edits.
        if not candidates:
            candidates = {w for w in edits2(word) if w in self.unigrams}
        
        return candidates if candidates else {word}

    def context_score(self, candidate, prev_word, next_word):
        score = 1.0
        epsilon = 1e-8  # to avoid zero probability
        if prev_word:
            score *= (self.bigrams.get((prev_word, candidate), 0) + epsilon)
        if next_word:
            score *= (self.bigrams.get((candidate, next_word), 0) + epsilon)
        score *= (self.unigrams.get(candidate, 0) + epsilon)
        return score

def process_line(line, model):
    words_list = line.strip().lower().split()
    corrected = []
    for i in range(len(words_list)):
        candidates = model.get_candidates(words_list[i])
        # Use previous corrected word and next original word as context.
        prev_word = corrected[i-1] if i > 0 else None
        next_word = words_list[i+1] if i < len(words_list)-1 else None

        best = words_list[i]
        best_score = 0
        for candidate in candidates:
            score = model.context_score(candidate, prev_word, next_word)
            if score > best_score:
                best = candidate
                best_score = score
        corrected.append(best)
    return ' '.join(corrected)

# Initialize and load data from all four files.
model = NGramModel()
model.load_data(
    '/kaggle/input/ngrams-data/bigrams (2).txt',
    '/kaggle/input/ngrams-data/fivegrams (2).txt',
    '/kaggle/input/ngrams-data/coca_all_links (2).txt',
    '/kaggle/input/bigdata/big.txt'
)

# Example usage
test_cases = [
    "corectud",
    "I am a studnet in the univercity",
    "Thiss is a goood example"
]

for line in test_cases:
    corrected = process_line(line, model)
    print(f"Input: {line}")
    print(f"Output: {corrected}\n")


Input: corectud
Output: corrected

Input: I am a studnet in the univercity
Output: i am a student in the university

Input: Thiss is a goood example
Output: this is a good example



## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

## Justifications for Implementation Choices

### 1. N-Gram Dataset Selection

#### Bigrams & Fivegrams:
- Bigrams capture local word dependencies (e.g., "student" often follows "university"), while fivegrams (split into overlapping bigrams) provide broader contextual patterns.
- This balances specificity and generality for better corrections.

#### COCA Corpus:
- The Corpus of Contemporary American English adds diverse, real-world language usage.
- Improves coverage of modern vocabulary and collocations.

#### big.txt:
- Retained for backward compatibility with Norvig’s baseline.
- Ensures common words are prioritized as in the original model.

---

### 2. Candidate Generation Strategy

#### Staged Edits (edits1 → edits2):
- Prioritizes one-edit candidates (e.g., "corectud" → "corrected") before two-edit candidates, reducing computational overhead.
- Keyboard adjacency edits (e.g., "studnet" → "student") are included in the first stage, as typos often involve adjacent keys.

#### Keyboard-Aware Edits:
- Explicitly models common typos caused by physical keyboard proximity (e.g., "m" ↔ "n").
- Addresses a limitation in generic edit-distance approaches.

---

### 3. Contextual Scoring

#### Bigram Context:
- Scores candidates using previous and next word bigrams (e.g., "I am a student" favors "student" over "students" if "am a" precedes it).
- This leverages syntactic and semantic context.

#### Smoothing (Epsilon):
- Adds epsilon = 1e-8 to avoid zero probabilities for unseen bigrams/unigrams.
- Ensures rare but valid candidates aren’t discarded.

---

### 4. Efficiency vs. Accuracy Trade-offs

#### No Beam Search:
- Processes words sequentially (left-to-right) using local context for speed.
- While beam search could optimize global sentence coherence, it would increase complexity and runtime.

#### Partial Context Use:
- Uses corrected previous words but original next words during correction.
- Balances accuracy and computational feasibility.

---

## Analysis of the Approach

### Why Context Matters:
- Norvig’s model corrects words in isolation, leading to errors like "there" → "three" in "There dog ran."
- Our model uses bigrams (e.g., "dog ran" favors "The" over "Three") to resolve such ambiguities.

### Keyboard Edits:
- Physical typos (e.g., "univercity" → "university") are better captured through adjacency-aware replacements than generic edits.

### Data Diversity:
- Combining multiple corpora (COCA, big.txt) reduces bias toward any single domain and improves robustness.

---

## Improvements Over Norvig’s Approach

### Context-Awareness:
- Uses bigrams to resolve ambiguities (e.g., "read the book" vs. "read the hook").
- This is impossible in Norvig’s unigram model.

### Keyboard-Adjacent Typos:
- Explicitly models common typo patterns.
- Improves precision for errors like "stuid" → "study".

### Corpus Blending:
- Integrates domain-specific data (COCA) with general text (big.txt) for broader coverage.

### Staged Correction:
- Balances efficiency (edits1 first) and fallback coverage (edits2 if needed).
- Unlike Norvig’s flat candidate generation, this ensures faster corrections with better accuracy.

---

## Limitations & Future Work

### Higher-Order N-Grams:
- Fivegrams could be used directly (not just split into bigrams) for richer context.
- However, sparsity may require smoothing techniques.

### Global Optimization:
- Beam search or Viterbi-like algorithms could improve sentence-level coherence.

---

This approach significantly advances Norvig’s model by integrating contextual, linguistic, and typographic insights, achieving higher accuracy in real-world scenarios.


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [15]:
###############################
# Norvig's Corrector Code (Context-Insensitive)
###############################

def words(text): 
    return re.findall(r'\w+', text.lower())

# Using big.txt from our big data file for Norvig's corrector
with open('/kaggle/input/bigdata/big.txt', 'r', encoding='utf-8') as f:
    big_text = f.read()
WORDS = Counter(words(big_text))
N = sum(WORDS.values())

def P(word): 
    "Probability of `word`."
    return WORDS[word] / N

def known(words_list): 
    "The subset of `words_list` that appear in the dictionary of WORDS."
    return set(w for w in words_list if w in WORDS)

def edits1_norvig(word):
    "All edits that are one edit away from `word`."
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2_norvig(word): 
    "All edits that are two edits away from `word`."
    return set(e2 for e1 in edits1_norvig(word) for e2 in edits1_norvig(e1))

def candidates_norvig(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1_norvig(word)) or known(edits2_norvig(word)) or [word])

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates_norvig(word), key=P)

def correct_sentence_norvig(sentence):
    return ' '.join(correction(word) for word in sentence.split())

In [16]:
import re
import random
from collections import defaultdict, Counter

###############################
# Evaluation Code
###############################

def introduce_noise(word, noise_prob):
    """Introduce a random noise operation on a word with probability noise_prob."""
    if random.random() < noise_prob:
        op = random.choice(['delete', 'insert', 'substitute', 'transpose'])
        if op == 'delete' and len(word) > 1:
            pos = random.randrange(len(word))
            return word[:pos] + word[pos+1:]
        elif op == 'insert':
            pos = random.randrange(len(word)+1)
            letter = random.choice('abcdefghijklmnopqrstuvwxyz')
            return word[:pos] + letter + word[pos:]
        elif op == 'substitute':
            pos = random.randrange(len(word))
            letter = random.choice('abcdefghijklmnopqrstuvwxyz')
            while letter == word[pos]:
                letter = random.choice('abcdefghijklmnopqrstuvwxyz')
            return word[:pos] + letter + word[pos+1:]
        elif op == 'transpose' and len(word) > 1:
            pos = random.randrange(len(word)-1)
            return word[:pos] + word[pos+1] + word[pos] + word[pos+2:]
    return word

def add_noise_to_sentence(sentence, noise_prob):
    """Apply noise to each word in the sentence."""
    return ' '.join(introduce_noise(word, noise_prob) for word in sentence.split())

def evaluate(corrector_func, test_set, noise_prob):
    """
    Evaluate a given corrector function on a test set.
    Returns the overall word-level accuracy.
    """
    total_words = 0
    correct_words = 0
    for original in test_set:
        noisy = add_noise_to_sentence(original, noise_prob)
        corrected = corrector_func(noisy)
        # Normalize and split into words.
        original_words = original.lower().split()
        corrected_words = corrected.lower().split()
        for ow, cw in zip(original_words, corrected_words):
            total_words += 1
            if ow == cw:
                correct_words += 1
    return correct_words / total_words if total_words > 0 else 0

# Example Test Set: You can expand this list for more comprehensive evaluation.
test_set = [
    "This is a test sentence for evaluation.",
    "The quick brown fox jumps over the lazy dog.",
    "I am a student in the university.",
    "This is a good example of a context-sensitive corrector.",
    "Our solution should outperform Norvig's corrector on noisy input.",
    "She sells seashells by the seashore.",
    "A journey of a thousand miles begins with a single step.",
    "To be or not to be, that is the question.",
    "All that glitters is not gold.",
    "The pen is mightier than the sword.",
    "An apple a day keeps the doctor away.",
    "Better late than never.",
    "Actions speak louder than words.",
    "Practice makes perfect.",
    "When in Rome, do as the Romans do.",
    "The early bird catches the worm.",
    "A picture is worth a thousand words.",
    "Beauty is in the eye of the beholder.",
    "Necessity is the mother of invention.",
    "Honesty is the best policy."
]

# Noise probability can be adjusted (e.g., 0.2, 0.3, 0.5)
noise_probability = 0.3

# Evaluate our corrector (context-sensitive)
our_accuracy = evaluate(lambda s: process_line(s, model), test_set, noise_probability)
# Evaluate Norvig's corrector (context-insensitive)
norvig_accuracy = evaluate(correct_sentence_norvig, test_set, noise_probability)

print(f"Noise Probability: {noise_probability*100:.0f}%")
print(f"Accuracy of our corrector: {our_accuracy*100:.2f}%")
print(f"Accuracy of Norvig's corrector: {norvig_accuracy*100:.2f}%")

Noise Probability: 30%
Accuracy of our corrector: 73.05%
Accuracy of Norvig's corrector: 70.92%


We expect the accuracy of our corrector to be better in real case scenarios, because here we did a completly random noise, and we will not get the benefite of the keyboard adjacent as it happen only in real cases when the user write on the keybord

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)